# Après Java 8
Ce notebook présente brièvement les nouveautés importantes de Java à partir de Java 8.
La suite sera illustrée sur un exemple simple composé d'une liste de personnes. 

In [1]:
%%shell
gitpuller https://github.com/ebpro/sample-person.git develop /home/jovyan/work/src/samples/sample-person
cd "/home/jovyan/work/src/samples/sample-person"
mvn -quiet -ntp clean package

[2022-10-27 16:38:18,242] INFO -- Repo /home/jovyan/work/src/samples/sample-person doesn't exist. Cloning...
[2022-10-27 16:38:18,979] INFO -- Repo /home/jovyan/work/src/samples/sample-person initialized


$ git clone --depth 1 --branch develop -- https://github.com/ebpro/sample-person.git /home/jovyan/work/src/samples/sample-person

Cloning into '/home/jovyan/work/src/samples/sample-person'...



In [2]:
List<String> addedJars = %jars "/home/jovyan/work/src/samples/sample-person/target/SamplePerson-*.jar";  
addedJars;

[/home/jovyan/work/src/samples/sample-person/target/SamplePerson-1.0-SNAPSHOT.jar]

In [3]:
import fr.univtln.bruno.samples.persons.Person;
List<Person> personnes = Arrays.asList(
                        Person.getInstance("pierre.durand@a.fr","Pierre", "Durand", 20), 
                        Person.getInstance("marie.durand@b.fr","Marie", "Durand", 14),
                        Person.getInstance("albert.martin@c.fr","Albert", "Martin", 30));
personnes;

[Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20), Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14), Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30)]

Avec une liste d’instances de la classe Personne(email, prénom, nom, age) et si la classe `Person` redéfini `equals` et `hashcode` et implante l'interface `Comparable` en fonction de l'email on peut trier canoniquement la liste par email.

In [4]:
Collections.sort(personnes);
personnes;

[Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30), Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14), Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20)]

Pour trier par un autre critère par exemple l'âge, on peut implanter l'interface Comparator et en fournir une instance. 

In [5]:
public class PersonComparatorByAge implements Comparator<Person> {
    public int compare(Person person1, Person person2) {
        return person1.getAge()-person2.getAge();
    }
}

Collections.sort(personnes, new PersonComparatorByAge());
personnes;

[Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14), Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20), Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30)]

Si le comparateur ne doit être utilisé qu'une fois on peut utiliser une classe anonyme.

In [6]:
//Tri par prénom, puis nom
Collections.sort(personnes, new Comparator<Person>() {
    @Override
    public int compare(Person o1, Person o2) {
        return (o1.getFirstName().equals(o2.getFirstName())?o1.getLastName().compareTo(o2.getLastName()):o1.getFirstName().compareTo(o2.getFirstName()));
    }
});
personnes;

[Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30), Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14), Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20)]

## Java 8

### Les lambdas expressions
Plus généralement, l'utilisation d'une interface à une seule méthode est un pattern classique en Java, c'est le cas par exemple lorsque l'on veut trier par un autre critère que le tri canonique ("par défaut") définit via l'interface Comparable\<T\> ou pour donner le code de réavction dans une IHM.

Les lambda expressions (en Java) sont un moyen de manipuler des instances d'interfaces à une seule méthode. 

A partir de Java 8 on peut utiliser des lambda expressions. Elles peuvent être vues comme des instances de classes anonymes n'ayant qu'une seule méthode dont les types de retour et des paramètres sont inférés.

Une lambda expression est composée :
  * d'un ou ou de plusieurs noms de paramètres (alors entre parenthèses) éventuellement typés
  * du symbole →
  * d'un corps de fonction, s’il est réduit à une expression le return et les accolades sont facultatifs.

Dans notre example, le tri de la liste par nom s'écrit alors :

In [7]:
Collections.sort(personnes, (o1, o2) -> o1.getLastName().compareTo(o2.getLastName()));
personnes;

[Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14), Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20), Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30)]

## Les interfaces fonctionnelles

Java 8 définit le concept d’interfaces fonctionnelles (elles ont extactement une méthode). Elles permettent de manipuler des références vers des lambda expressions ou des méthodes. Une interface peut être définie comme fonctionnelle avec l’annotation @FunctionalInterface. Un ensemble d’interfaces fonctionnelles classiques est proposé dans le JDK :

  * `Function<T,R>` - takes an object of type T and returns R.
  * `Supplier<T>` - just returns an object of type T.
  * `Predicate<T>` - returns a boolean value based on input of type T.
  * `Consumer<T>` - performs an action with given object of type T.
  * `BiFunction<T,U>` - like Function but with two parameters.
  * `BiConsumer<T,U>` - like Consumer but with two parameters.
  * ... https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/function/package-summary.html
  
Nous allons voir des exemples avec quatre interfaces courament utilsées : `Function`, `Predicate`, `Supplier` et `Consumer`.

L'interface [Function](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/function/Function.html) applique un traitement sur son unique paramètre. 

Il est par exemple possible de définir une variable `at` qui contient une référence vers une "fonction" avec un seul paramètre ici de type String qui ajoute un `@` en préfixe à une chaîne de caractères (donc avec comme type de retour String). Le nom de l'unique méthode d'une `Function` est `apply`.

In [8]:
import java.util.function.Function;
Function<String, String> at = name -> "@" + name;

On peut alors définir des méthodes qui prennent en paramètre une instance d'une interface fonctionnelle. 

Ici une méthode qui parcourt la liste de personnes, applique la Function sur le prénom et l'affiche.

In [9]:
public class AfficheurDePersonnes {
    public static void appliqueEtAfficher(Function<String,String> fonction) {
        for (Person p : personnes) System.out.println(fonction.apply(p.getFirstName()));
    }
}    

AfficheurDePersonnes.appliqueEtAfficher(at);

@Marie
@Pierre
@Albert


On peut aussi directement utiliser une lambda de types compatibles (noter que les types de retour et du paramètre `s` sont inférés à partir des paramètres de l'interface fonctionnelle attendue). 

In [10]:
AfficheurDePersonnes.appliqueEtAfficher(s->s.toUpperCase());

MARIE
PIERRE
ALBERT


Les [Supplier](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/function/Supplier.html) sont des fonctions sans paramètre qui produise une valeur et possède donc un seul paramètre celui de l'objet retourné. C'est une sorte de factory. Ainsi `Supplier<String>` fournis des chaines de caratères, `Supplier<Integer>` des objets représentant des entiers, ... . Pour les primitifs, il existe des variantes dédiées de l'interface (donc sans paramètres) comme [IntSupplier](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/function/IntSupplier.html).

On peut ainsi par exemple, définir deux fonctions qui affichent 10 entiers à partir de suppliers (Object ou primitif).

In [11]:
import java.util.function.Supplier;
import java.util.function.IntSupplier;
import java.util.Random;

public class AfficheurDeNombre {
    private static int nb = 10;
    public static void afficher(Supplier<Double> supplier) {
        for (int i=0;i<nb;i++) System.out.print("["+supplier.get()+"]"+(i<nb-1?" ":"\n"));        
    }
    public static void afficher(IntSupplier supplier) {
        for (int i=0;i<nb;i++) System.out.print("("+supplier.getAsInt()+")"+(i<nb-1?",":"\n"));
    }
} 

et les utiliser avec des lamdas conforme à l'interface Supplier.

In [12]:
Random random = new Random();
AfficheurDeNombre.afficher(()->random.nextDouble(6)+1);
AfficheurDeNombre.afficher(()->Integer.valueOf(random.nextInt(50)+1)*2);

[1.5668726986095152] [6.608568729556655] [5.52967612239488] [3.9849903718134203] [1.4453343329004835] [2.240759521499399] [5.757127545928561] [1.8146327952736356] [4.232440148650268] [1.3910058334113669]
(32),(6),(98),(80),(14),(20),(20),(76),(16),(74)


L'opérateur `::` permet de récupérer une référence vers un constructeur, une méthode statique ou d'instance existante que l'on pourra manipuler à travers une interface fonctionnelle.

On peut ainsi définir une méthode qui ajoute des Personnes à une liste qui est créée à partir d'un Supplier qui fournit une liste de Personne en paramètre.

In [13]:
public void testSupplier(Supplier<List<Person>> plFactory) {
  List <Person> pl = plFactory.get();
  pl.add(Person.getInstance("pierre.durand@a.fr","Pierre", "Durand", 20)); 
  pl.add(Person.getInstance("marie.durand@b.fr","Marie", "Durand", 14));
  System.out.println(pl.getClass()+ "->" +pl);
}

Il est possible de passer uen référence de type Supplier :

In [14]:
Supplier<List<Person>> personListFactory = ArrayList::new;
testSupplier(personListFactory);

class java.util.ArrayList->[Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20), Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14)]


oui directement la référence :

In [15]:
testSupplier(LinkedList::new);

class java.util.LinkedList->[Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20), Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14)]


L'interface `Consumer` a un paramètre, le type d'objet consommé et aucun type de retour.

L'interface `Predicate` a un paramètre, le type d'objet sur lequel s'applique le prédicat et renvoie un booléen.

### Applications à la liste de personnes
Si l'on chercher à afficher une sous liste de personnes qui vérifient un prédicat, il est possible de faire en créant une classe générique qui applique un prédicat sur un objet que l'on peut parcourir ([Iterable](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/Iterable.html) :

In [16]:
import java.util.function.Predicate;
public class Processor<T> {
    public List<T> find(Iterable<T> iterable, Predicate<T> predicate,  Consumer<T> consumer) {
        List<T> list = new ArrayList<>();
        for (T t : iterable)
            if (predicate.test(t))
                list.add(t);
        return list;
    }
}

et utiliser une lambda expression pour le prédicat et une référence vers une fonction pour afficher le résultat

In [17]:
new Processor<Person>().find(personnes,
                         p -> p.getLastName().equals("Durand")
                            && p.getAge() >= 10
                            && p.getAge() <= 15,                    
                         System.out::println);

[Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14)]

Le même Processor peut-être utilisé pour afficher uniquement des entiers pairs (Attention ici vu comme des Objets et non des primitifs).

In [18]:
new Processor<Integer>().find(Arrays.asList(2,6,7,8,5),n->n % 2==0,System.out::println);

[2, 6, 8]

## Les Streams

Il est aussi très courant de devoir traiter des séquences d'objets ou de primitifs. Pour cela, Java 8 introduit la notion de [Stream](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/stream/package-summary.html). Un stream ne stocke pas la séquence mais permet de la traiter à travers un pipeline (éventuellement parallèle). La construction et le traitement des stream peut être “lazzy”. Les Stream sont conçu pour être utilisés avec une approche fonctionnelle.

Un stream peut par exemple être créé au dessus d’une collection (avec la méthode `.stream()`), à partir d'un tableau (`Arrays.stream(Object[])`), d'un fichier (` BufferedReader.lines()`), de diverses factories (
[Stream.of(T...)](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/stream/Stream.html#of(T...)),
[IntStream.range(int, int)](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/stream/IntStream.html#range(int,int)), 
et de nombreuses autres méthodes.

Un Consummer peut être appliquer à chaque élément d'un stream [foreach](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/stream/Stream.html#forEach(java.util.function.Consumer).

Afficher une séquence d'entiers.

In [19]:
import java.util.stream.IntStream;
IntStream.range(10,15).forEach(System.out::println);

10
11
12
13
14


ou une liste de personnes

In [20]:
personnes.stream().forEach(System.out::println);

Person(email=marie.durand@b.fr, firstName=Marie, lastName=Durand, age=14)
Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20)
Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30)


ou les lignes d'un fichier

In [21]:
import java.util.stream.Stream;
try (Stream stream = Files.lines(Paths.get("/etc/legal"))) {
            stream.forEach(System.out::println);
}


The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.



ou les flux d'entrée/sortie (I/O stream)

In [22]:
try (FileReader fileReader = new FileReader("/etc/legal");
  BufferedReader br = new BufferedReader(fileReader)) {
  br.lines().forEach(System.out::println);
}


The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.



Un stream peut aussi être filtré en appliquant un Predicate :

In [23]:
personnes.stream().filter(p -> p.getAge() > 18).forEach(System.out::println);

Person(email=pierre.durand@a.fr, firstName=Pierre, lastName=Durand, age=20)
Person(email=albert.martin@c.fr, firstName=Albert, lastName=Martin, age=30)


Il est aussi possible d'appliquer une Function à chaque élément du flux avec `map()`.

Par exemple pour transformer un `Stream<Person>` en `Stream<String>` pour afficher les noms formattés des personnes de plus de 18 ans.

In [24]:
personnes.stream()
    .filter(p -> p.getAge() > 18)
    .map(p-> String.format("%s %s",p.getLastName().toUpperCase(), p.getFirstName().toLowerCase()) )
    .forEach(System.out::println);

DURAND pierre
MARTIN albert


Il est aussi possible de passer d'un flux d'objet à un flux de primitif et d'utiliser des références vers une méthode.
De plus, un ensemble d'opération dites d'aggrégations (disposant d'un systeme d'itération interne) par exemple : average, sum, min, max, count

In [25]:
double ageMoyen = personnes.stream()
    .mapToInt(Person::getAge)
    .average()
    .getAsDouble();
ageMoyen;

21.333333333333332

### Exception
Si une exception doit être testée dans une interface fonctionnelle cela alourdit fortement le code.

In [26]:
//Production de 10 entiers à raison d'un par 1/10s et calcul de la somme.
IntStream.range(0, 10)
        .map(i -> {try {
                        Thread.sleep(100);
                        } 
                   catch (java.lang.InterruptedException e) {}; 
                   return i*2;})
        .sum();

90

Une solution élégante est d'écrire une classe wrapper qui gère l'exception par exemple en l'encapsulant dans une runtime exception cf. https://www.baeldung.com/java-lambda-exceptions.

### Aggregation 
Il est existe aussi deux solutions très générales pour aggréger un Stream avec les mé [reduce() et collect()](https://docs.oracle.com/javase/tutorial/collections/streams/reduction.html) mais elles peuvent être complexes en première approche.

Dans la majorité des cas, les méthodes utilitaires proposées dans la classe [Collectors](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Collectors.html) remplissent les besoins courants :

#### Production d'une collection à partir d'un Stream :

In [27]:
import java.util.stream.Collectors;

List<String> AdultNames = personnes.stream()
    .filter(p -> p.getAge() > 18)
    .map(p-> String.format("%s %s",p.getLastName().toUpperCase(), p.getFirstName().toLowerCase()) )
    .collect(Collectors.toList());

AdultNames;

[DURAND pierre, MARTIN albert]

#### Opérations de groupement : 

In [28]:
//Regrouper les prénoms par nom dans une Map.
Map<String, List<String>> PersonfirstnamesGroupedByNames = personnes.stream()
        .collect(
            Collectors.groupingBy(
                Person::getLastName,                      
                Collectors.mapping(
                    Person::getFirstName,
                    Collectors.toList())));
PersonfirstnamesGroupedByNames;

{Durand=[Marie, Pierre], Martin=[Albert]}

#### Concaténation en String :

In [29]:
personnes.stream()
    .filter(p -> p.getAge() > 18)
    .map(p-> String.format("%s %s",p.getLastName().toUpperCase(), p.getFirstName().toLowerCase()) )
    .collect(Collectors.joining(",","-->","<--"));

-->DURAND pierre,MARTIN albert<--

Il est aussi possible de traiter les stream en parallèle mais il faut alors prendre en compte le cout de gestion des Thread.

Dans les exemples ci-dessous une pause artificielle est ajoutée pour favoriser le parallelisme.
cf. https://www.baeldung.com/java-when-to-use-parallel-stream 

In [30]:
Random random = new Random();
double ageMoyen;
long startTime = System.nanoTime();
ageMoyen =  IntStream.range(0, /* limit */ 20) //TRICK TO KNOW THE SIZE OF THE STREAM
        .mapToObj(unused -> {try {Thread.sleep(100);} catch (java.lang.InterruptedException e) {};
                             return Person.getInstance("pierre.durand@a.fr","Pierre", "Durand", random.nextInt(100));})
        .mapToInt(Person::getAge)
        .average()
        .getAsDouble(); //Il s'agit d'un Optionnal
long endTime = System.nanoTime();
System.out.println("moyenne calculée en "+(endTime - startTime)/1000000+"ms");

moyenne calculée en 2334ms


In [31]:
Random random = new Random();
double ageMoyen;
long startTime = System.nanoTime();
ageMoyen =  IntStream.range(0, /* limit */ 20) //TRICK TO KNOW THE SIZE OF THE STREAM 
        .unordered().parallel() //EXECUTION PARALELLE
        .mapToObj(unused -> {try {Thread.sleep(100);} catch (java.lang.InterruptedException e) {};
                             return Person.getInstance("pierre.durand@a.fr","Pierre", "Durand", random.nextInt(100));})
        .mapToInt(Person::getAge)
        .average()
        .getAsDouble(); //Il s'agit d'un Optionnal
long endTime = System.nanoTime();
System.out.println("moyenne calculée en "+(endTime - startTime)/1000000+"ms");

moyenne calculée en 442ms


Les streams utilisent le mécanismes sous-jacent de Java [ForkJoin](https://docs.oracle.com/javase/tutorial/essential/concurrency/forkjoin.html).

In [32]:
ForkJoinPool pool = new ForkJoinPool();
System.out.println("Parallelism = " +pool.getParallelism()); 

Parallelism = 8


## Les interfaces en général
A partir de Java 8, il est possible de définir une implantation par défaut dans une interface ainsi que des méthodes statiques.

In [33]:
public interface ActionClass<T> {
    //Depuis Java 9 les méthodes statiques sont autorisées
    static String getInterfaceDescription() {return "Une interface qui impose de définir une action et permet de l'exécuter une liste de <T>";}
    
    void action(T t);
    
    default void action(List<T> list) {
        for(T t:list) action(t);
    }
}

//Une classe qui affiche une String en majuscule et s'applique automatiquement à une liste de String
public class UpperCaser implements ActionClass<String> {
    public void action(String s) {System.out.println(s.toUpperCase());}
}
//Une classe qui affiche la concaténation du nom et du prénom d'une personne et s'applique automatiquement à une liste de Personne
public class PersonFullname implements ActionClass<Person> {
    public void action(Person p) {System.out.println(p.getFirstName()+" "+p.getLastName());}
}


//La factory .of() est apparue dans Java9 pour créer des Collections non mutables.
new UpperCaser().action(List.of("abc","def"));

new PersonFullname().action(personnes);

System.out.println(ActionClass.getInterfaceDescription());

ABC
DEF
Marie Durand
Pierre Durand
Albert Martin
Une interface qui impose de définir une action et permet de l'exécuter une liste de <T>


## Optional<T>
Pour traiter plus simplement les possibles tests des cas de valeurs nulles, Java8 introduit le type `Optional`, un conteneur qui fournit des méthodes utilitaires dédiées.
    
La factory `of` vérifie que la référence n'est pas nulle et provoque une exception à la création sinon. 
    
~~La méthode `get()` retourne la valeur.~~ Depuis Java 9, on préfére l'utilise de `orElseThrow()` qui émet une `NoSuchElementException` en l'abscence de valeur.

In [34]:
String str1 = "value1";
Optional<String> optional1 = Optional.of(str1);
optional1.orElseThrow();

value1

In [35]:
String str1Bis = null; // Provoque une Exception
Optional<String> optional1 = Optional.of(str1Bis);

EvalException: null

Il est possible de manipuler des références éventuellement nulles avec la factory `ofNullable()`.
Dans ce cas `OrElse()` est une alternative à pour obtenir une valeur par défault en cas de valeur nulle au lieu d'un Exception.

In [36]:
String str2 = "value2";
Optional<String> optional2 = Optional.ofNullable(str2);
optional2.orElseThrow();

value2

In [37]:
String str3 = null;
Optional<String> optional3 = Optional.ofNullable(str3);
optional3.orElseThrow();

EvalException: No value present

In [38]:
String str3Bis = null;
Optional<String> optional3Bis = Optional.ofNullable(str3Bis);
optional3Bis.orElse("defaultValue3");

defaultValue3

`OrElseGet(Supplier s)` est une variante plus efficace si le calcul de la valeur par défaut est complexe car alors la fonction n'est évaluée qu'en cas de valeur nulle.

Dans l'exemple ci dessous en cas de valeur nulle, une entrée alétoire du tableau est retournée.

In [39]:
String str4 = null;
//String str4 = "value3"; 
String[] values={"a1","b2","c3"};

Optional<String> optional4 = Optional.ofNullable(str4);

optional4.orElseGet( () -> values[new Random().nextInt(values.length)] );

c3

## Java 9 
### Modularisation

Java 9 introduit la notion de module. Un module regroupe des classes, permet de définir des dépendances entre modules et défini une API publique. Le JDK devient modulaire et les programmes utilisateurs peuvent l'être cela permet d'executer des programmes en consommant moins de mémoire. Les classes internes du JDK ne sont plus accessibles depuis l'application pour améliorer la sécurité. 

Plus de détails : https://openjdk.java.net/projects/jigsaw/quick-start

## Process
L'accès aux informations des processus systèmes est amélioré.
Les examples suivants illustre l'accès au processus de la JVM et à tous ceux du système.

In [40]:
ProcessHandle processHandle = ProcessHandle.current();
ProcessHandle.Info processInfo = processHandle.info();

System.out.println("PID: " + processHandle.pid());
System.out.println("Arguments: " + processInfo.arguments());
System.out.println("Command: " + processInfo.command());
System.out.println("Instant: " + processInfo.startInstant());
System.out.println("Total CPU duration: " + processInfo.totalCpuDuration());
System.out.println("User: " + processInfo.user());

PID: 123
Arguments: Optional[[Ljava.lang.String;@5d08d44c]
Command: Optional[/home/jovyan/.sdkman/candidates/java/19-tem/bin/java]
Instant: Optional[2022-10-27T16:38:11.240Z]
Total CPU duration: Optional[PT44.08S]
User: Optional[jovyan]


In [41]:
import java.util.stream.Stream;

Stream<ProcessHandle> liveProcesses = ProcessHandle.allProcesses();

liveProcesses.filter(ProcessHandle::isAlive).limit(3)
        .forEach(ph -> {
            System.out.println("PID: " + ph.pid());
            System.out.println("  Start: " + ph.info().startInstant());
            System.out.println("  User: " + ph.info().user());
        });

PID: 1
  Start: Optional[2022-10-27T16:37:26.730Z]
  User: Optional[jovyan]
PID: 7
  Start: Optional[2022-10-27T16:37:26.900Z]
  User: Optional[jovyan]
PID: 123
  Start: Optional[2022-10-27T16:38:11.240Z]
  User: Optional[jovyan]


//TODO : ProcessBuilder (Since Java 7)

## Inférence du type des variables locales
Depuis Java 10, le type des variables locales avec initialisation peut être inféré à la compilation avec le mot clé `var`.

In [42]:
var i = 3;
var myMap = new HashMap<Integer,String>();

myMap.put(1,"A");
myMap.put(2,"B");
myMap;

{1=A, 2=B}

Depuis Java 11, c'est possible dans les lambdas même si le type des paramètre est déjà inféré.
L'un des avantages est de pouvoir associer des annotations sans avoir à spécifier le type.

In [43]:
public @interface MyAnnotation { }

import java.util.function.BiFunction;
BiFunction<String,String,String> f1 = (s1, s2) -> s1 + s2;

BiFunction<String,String,String> f2 = (@MyAnnotation var s1, @MyAnnotation var s2) -> s1 + s2;

## Instance Of
Depuis Java 12 en preview, l'opérateur `instance of` permet de faire implicement un cast en cas de succès.

In [44]:
Object obj = Person.getInstance("pierre.durand@a.fr","Pierre", "Durand", 20);

if (obj instanceof Person p) {
    System.out.println(p.getFirstName());
}

Pierre


## Text blocks
Depuis Java 13, il est possible d'utiliser des litéraux multilignes pour les String.

In [45]:
String person1Json="""
{
    "fistname" : "Pierre",
    "lastname" : "Durand",
    "description" : "lorem ipsum.............\
..................\
..................lorem ipsum"
}    
""";
    
person1Json;

{
    "fistname" : "Pierre",
    "lastname" : "Durand",
    "description" : "lorem ipsum.................................................lorem ipsum"
}


### Records
Les `record` (depuis Java 14/15) permettent de définir très simplement des classes non mutables avec constructeurs, accesseurs (sans le préfixe get), toString().

In [46]:
public record Dog(int id, String name){ };

Dog d1 = new Dog(1, "Rex");
Dog d2 = new Dog(2, "Médor");
Dog d3 = new Dog(1, "Rex");

System.out.println(d1+"-> nom:"+d1.name());

Dog[id=1, name=Rex]-> nom:Rex


ainsi qu'une implantation de `equals` et `hashcode` (égalités de tous les composants). 
Il sont typement très utilisés pour les design pattern comme le DTO.

In [47]:
import java.util.stream.Collectors;
System.out.println(Stream.of(d1,d2,d3).collect(Collectors.toCollection(HashSet::new)).toString());            

[Dog[id=2, name=Médor], Dog[id=1, name=Rex]]


Il est possible de redéfinir ou de surcharger le constructeur et les accesseurs/modificateurs et d'ajouter des variables et méthodes de classes **mais pas de variable d'instance**.

In [48]:
public record User(int id, String firstname){ 
        public User {
            if(id < 0) { throw new IllegalArgumentException("Id has to be greater than 0.");
        }
    }
};

new User(-1,"Pierre");

EvalException: Id has to be greater than 0.

## Sealed and Hidden classes

```Java
public final class B extends A  { };
public non-sealed class C extends A  { };

public abstract sealed class A
    permits B, C {
}
```

### Switch

A partir de Java 17 le switch peut évaluer une expression et retourner sa valeur.

In [49]:
public enum Day { SUNDAY, MONDAY, TUESDAY,
    WEDNESDAY, THURSDAY, FRIDAY, SATURDAY; }

Day day = Day.WEDNESDAY;    
//Day day = Day.TUESDAY;    

int l = 
        switch (day) {
            case MONDAY, FRIDAY, SUNDAY -> 6;
            case TUESDAY                -> 7;
            case THURSDAY, SATURDAY     -> 8;
            case WEDNESDAY              -> 9;
            default -> throw new IllegalStateException("Invalid day: " + day);
        };

l;

9

et utiliser des Pattern de classes.

In [50]:
Dog d1 = new Dog(1, "Rex");
User u1= new User(1,"Pierre");

public String check(Object o) {
    return switch (obj) {
                case User u -> "I'm a user "+u.firstname();
                case Dog d -> "I'm a dog "+d.name();
                case null -> "NULL !";
                default -> "A simple object.";
    };
}

System.out.println("-->"+check(d1));

EvalException: Preview features are not enabled for REPL/$JShell$161 (class file version 63.65535). Try running with '--enable-preview'

### Record Pattern

En preview dans Java 19.

In [51]:
record Animal(int id, String name){}
record Budy(long id, String firstname){}

static void printName(Object o) {
    if (o instanceof Animal(var id, var name) d)
        System.out.printf("dog %d name is %s.",d.id(),d.name());
    else 
        if (o instanceof Budy(var id, var firstname) d)
            System.out.printf("Budy %d firstname is %s.",d.id(),d.firstname());
}
    
printName(new Animal(3,"Félix"));
printName(new Budy(1L,"Pierre"));

EvalException: Preview features are not enabled for REPL/$JShell$165 (class file version 63.65535). Try running with '--enable-preview'